# <center>为什么需要量化</center>

## 1. 什么是量化

我们都知道，不论是那种深度学习框架训练好模型，它的权重文件一般都是FP32（**单精度浮点**）类型的。在深度学习训练和推理的过程中，最常用的也是单精度浮点即FP32。当然也会有FP64、FP16、BF16、TF32等更多的精度：


<center>
    
![图片](./pictures/FP32与FP16精度差异.png "FP32与FP16精度差异")
</center>

<center>FP32与FP16精度差异</center>
<!-- <div align=center><img src="./pictures/FP32与FP16精度差异.png"><title "haibai"></div> -->

FP32是单精度浮点数，用8bit位表示指数，23bit位表示小数；FP16是半精度浮点数，用5bit位表示指数，10bit位表示小数；BF16是对FP32单精度浮点数截断数据，即8bit位表示指数，7bit位表示小数；TF32 是一种截短的 Float32 数据格式，将 FP32 中 23 个尾数位截短为 10 bits，而指数位仍为 8 bits，总长度为 19 (=1 + 8 + 10)  bits。

对于浮点数来说，指数位表示该类型数据可达的动态范围，而尾数位表示精度。FP16的普遍精度是~5.96e−8 (6.10e−5) … 65504，而我们模型中的FP32权重有部分数值是**1e-10**级别。这样从FP32->FP16会导致部分精度丢失，从而模型的精度也会下降一些。

<center>
    
![图片](./pictures/两种模式运算.png)
</center>

<center>两种模式运算-在Ampere-tensor-cores</center>

其实从FP32->FP16也是一种量化，只不过因为FP32->FP16几乎是无损的(**CUDA中使用__float2half直接进行转换**)，不需要calibrator去校正、更不需要retrain。

而且FP16的精度下降对于大部分任务影响不是很大，甚至有些任务会提升。NVIDIA对于FP16有专门的Tensor Cores可以进行矩阵运算，相比FP32来说吞吐量直接提升一倍，提速效果明显！

<center>
    
![图片](./pictures/量化简单示意图.png)
</center>

<center>量化简单示意图</center>

实际点来说，**量化就是将我们训练好的模型，不论是权重，还是计算OP，都转换为低精度去计算**。因为FP16的量化很简单，所以实际中我们谈论的量化更多是INT8量化，当然也有3bit、4bit的量化，不过目前来说比较常见比较实用的，也就是INT8量化了，接下来的都是INT8量化。

经过INT8量化后的模型：

- 模型容量变小了，这个很好理解，**FP32的权重变成INT8，大小直接缩放了4倍**；
- 模型运行速度可以提升，实际卷积计算的OP是INT8类型，在特定硬件下可以利用INT8的指令集去实现高吞吐，**不论是GPU还是Intel、ARM等平台都有INT8指令集优化**；
- 对于某些设备，使用INT8的模型耗电量更少，对于嵌入式端侧设备来说是有很大的提升的。

所以说，随着我们模型越来越大，需求越来越高，模型的量化自然是少不了的一项技术。

**如果你担心INT8量化对于精度的影响，我们可以看下NVIDIA量化研究的一些结论：**

<center>
    
![图片](./pictures/FP32-INT8量化指标评测.png)
</center>

<center>FP32-INT8量化指标评测</center>

该量化评测结果出自《INTEGER QUANTIZATION FOR DEEP LEARNING INFERENCE: PRINCIPLES AND EMPIRICAL EVALUATION》。

## 2. 量化的发展现状

量化技术已经广泛应用于实际生产环境了，也有很多的大厂开源了其量化方法。不过比较遗憾的是目前这些方法比较琐碎，没有一套比较成熟、完善的量化方案，使用起来稍微有点难度。不过我们仍然可以从这些框架中学习到很多。

### Google

谷歌是比较早进行量化尝试的大厂了，感兴趣的可以看下Google的白皮书`Quantizing deep convolutional networks for efficient inference: A whitepaper`以及`Quantization and Training of Neural Networks for Efficient Integer-Arithmetic-Only Inference`。

TensorFlow很早就支持了量化训练，而TFLite也很早就支持了后训练量化，感兴趣的可以看下TFLite的量化规范。目前TensorRT也支持TensorFlow训练后量化导出的模型。

### TensorRT

TensorRT在2017年公布了自己的后训练量化方法，不过没有开源，NCNN按照这个思想实现了一个，也特别好用。不过目前TensorRT8也支持直接导入通过ONNX导出的（量化感知训练，QAT）好的模型，使用上方便了不少。

<center>
    
![图片](./pictures/图灵显卡上的INT8速度.png)
</center>

<center>图灵显卡上的INT8速度</center>

NVIDIA自家也推出了针对Pytorch的量化工具（为什么没有TensorFlow，因为TF已经有挺好用的官方工具了），支持PTQ以及QAT，称为Pytorch Quantization，之后也会提到。

### TVM

TVM有自己的INT8量化操作，可以跑量化，我们也可以添加自己的算子。不过TVM目前只支持PTQ，可以通过交叉熵或者percentile的方式进行校准。不过如果动手能力强的话，应该可以拿自己计算出来的scale值传入TVM去跑，应该也有人这么做过了。

比较有参考意义的一篇文章：

- [ViT-int8 on TVM：提速4.6倍，比TRT快1.5倍](https://zhuanlan.zhihu.com/p/365686106)。

当然，还有很多优秀的量化框架，限于篇幅就不多说啦，想看详细的可以看[这篇文章](https://zhuanlan.zhihu.com/p/355598250)，后续如果涉及到具体知识点也会再提到。